# Flat Representations Tutorial

In [1]:
import sys

sys.path.append("../")

In [2]:
# Imports
import os
from pathlib import Path
import polars as pl, numpy as np
import polars.selectors as cs

from EventStream.data.dataset_polars import Dataset
from EventStream.evaluation.FT_task_baseline import load_flat_rep, fit_baseline_task_model

from sklearn.ensemble import RandomForestClassifier

In [3]:
ESD = Dataset.load(Path(os.getcwd()) / "processed/sample")

In [4]:
ESD.cache_flat_representation(
    subjects_per_output_file=None,
    feature_inclusion_frequency=None,
    do_overwrite=False,
    do_update=True,
)
flat_reps = load_flat_rep(ESD, window_sizes=['7d', '30d', 'FULL'])
display(flat_reps['train'].head().collect())

Flattening Splits:   0%|          | 0/3 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/1 [00:00<?, ?it/s]

subject_id,timestamp,static/eye_color/BLUE,static/eye_color/BROWN,static/eye_color/GREEN,static/eye_color/HAZEL,static/eye_color/UNK,7d/HR/HR/count,7d/HR/HR/has_values_count,7d/age/age/count,7d/age/age/has_values_count,7d/department/CARDIAC/count,7d/department/ORTHOPEDIC/count,7d/department/PULMONARY/count,7d/department/UNK/count,7d/lab_name/GCS__EQ_1/count,7d/lab_name/GCS__EQ_1/has_values_count,7d/lab_name/GCS__EQ_10/count,7d/lab_name/GCS__EQ_10/has_values_count,7d/lab_name/GCS__EQ_11/count,7d/lab_name/GCS__EQ_11/has_values_count,7d/lab_name/GCS__EQ_12/count,7d/lab_name/GCS__EQ_12/has_values_count,7d/lab_name/GCS__EQ_13/count,7d/lab_name/GCS__EQ_13/has_values_count,7d/lab_name/GCS__EQ_14/count,7d/lab_name/GCS__EQ_14/has_values_count,7d/lab_name/GCS__EQ_15/count,7d/lab_name/GCS__EQ_15/has_values_count,7d/lab_name/GCS__EQ_2/count,7d/lab_name/GCS__EQ_2/has_values_count,7d/lab_name/GCS__EQ_3/count,7d/lab_name/GCS__EQ_3/has_values_count,7d/lab_name/GCS__EQ_4/count,7d/lab_name/GCS__EQ_4/has_values_count,7d/lab_name/GCS__EQ_5/count,7d/lab_name/GCS__EQ_5/has_values_count,…,FULL/lab_name/GCS__EQ_8/min,FULL/lab_name/GCS__EQ_9/min,FULL/lab_name/SOFA__EQ_1/min,FULL/lab_name/SOFA__EQ_2/min,FULL/lab_name/SOFA__EQ_3/min,FULL/lab_name/SOFA__EQ_4/min,FULL/lab_name/SpO2/min,FULL/lab_name/UNK/min,FULL/lab_name/creatinine/min,FULL/lab_name/potassium/min,FULL/temp/temp/min,FULL/HR/HR/max,FULL/age/age/max,FULL/lab_name/GCS__EQ_1/max,FULL/lab_name/GCS__EQ_10/max,FULL/lab_name/GCS__EQ_11/max,FULL/lab_name/GCS__EQ_12/max,FULL/lab_name/GCS__EQ_13/max,FULL/lab_name/GCS__EQ_14/max,FULL/lab_name/GCS__EQ_15/max,FULL/lab_name/GCS__EQ_2/max,FULL/lab_name/GCS__EQ_3/max,FULL/lab_name/GCS__EQ_4/max,FULL/lab_name/GCS__EQ_5/max,FULL/lab_name/GCS__EQ_6/max,FULL/lab_name/GCS__EQ_7/max,FULL/lab_name/GCS__EQ_8/max,FULL/lab_name/GCS__EQ_9/max,FULL/lab_name/SOFA__EQ_1/max,FULL/lab_name/SOFA__EQ_2/max,FULL/lab_name/SOFA__EQ_3/max,FULL/lab_name/SOFA__EQ_4/max,FULL/lab_name/SpO2/max,FULL/lab_name/UNK/max,FULL/lab_name/creatinine/max,FULL/lab_name/potassium/max,FULL/temp/temp/max
u8,datetime[μs],bool,bool,bool,bool,bool,i64,i64,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,2010-06-24 13:23:00,null,null,true,null,null,4,4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,-0.36323,null,-0.312384,null,-1.512411,1.294228,-0.558276,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.044146,null,-0.312384,null,-1.407393
0,2010-06-24 14:23:00,null,null,true,null,null,5,5,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,-0.36323,null,null,null,-1.512411,1.294228,-0.55825,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.044146,null,null,null,0.167901
0,2010-06-24 15:23:00,null,null,true,null,null,7,5,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,3.4028e38,null,null,-0.36323,null,null,null,-1.512411,null,-0.558224,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-3.4028e38,null,null,0.044146,null,null,null,1.533159
0,2010-06-24 16:23:00,null,null,true,null,null,9,7,4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,3.4028e38,null,null,null,-0.36323,null,null,-0.399881,-1.512411,1.585936,-0.558199,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-3.4028e38,null,null,null,1.673651,null,null,-0.399881,1.585668
0,2010-06-24 17:23:00,null,null,true,null,null,12,9,5,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,null,null,null,null,null,null,-0.36323,null,null,-0.399881,-1.512411,1.585936,-0.558173,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.67365

In [5]:
task_df = (
    ESD.events_df
    .groupby('subject_id')
    .agg(pl.col('timestamp').sample().first().alias('end_time'))
    .with_columns(
        pl.lit(np.random.choice([0, 1, 2], size=len(ESD.subject_ids))).alias('label'),
        pl.lit(None, dtype=pl.Datetime).alias('start_time')
    )
).lazy()

Loading events from /home/mmd/Projects/EventStreamGPT/sample_data/processed/sample/events_df.parquet...
